In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name = "Search Agent",
    tools = [WebSearchTool(search_context_size="low")],
    instructions=INSTRUCTIONS,
    model = "gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")  #By this be make it mandatory for the agent to run the specified tool
)

In [4]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of June 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems:

- **LangGraph**: An extension of LangChain, LangGraph introduces a graph-based system for managing agent workflows, supporting branching, looping, and conditional flows. It's particularly useful for applications requiring complex decision-making processes. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen automates the generation of code, models, and processes for complex workflows. It leverages large language models (LLMs) to assist developers in building, fine-tuning, and deploying AI solutions with minimal manual coding. ([blog.mechcloud.io](https://blog.mechcloud.io/top-5-ai-agent-frameworks-in-2025?utm_source=openai))

- **CrewAI**: This framework orchestrates role-based AI agents for collaborative tasks, focusing on simplicity and minimal setup requirements. It's popular for customer service and marketing automation, allowing agents to work together on shared objectives. ([datacamp.com](https://www.datacamp.com/blog/best-ai-agents?utm_source=openai))

- **Eliza**: A Web3-friendly AI agent operating system, Eliza integrates seamlessly with blockchain applications, enabling the deployment of AI agents in decentralized environments. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

- **AutoAgent**: A fully automated, zero-code framework for LLM agents, AutoAgent enables users to create and deploy AI agents through natural language alone, making AI development accessible without programming skills. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, catering to diverse applications from complex decision-making to decentralized systems and user-friendly development environments. 

In [5]:
HOW_MANY_SEARCHES = 2

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query")

    query: str = Field(description="The search term to use for the web search")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

planner_agent = Agent(
    name = "PlannerAgent",
    instructions=INSTRUCTIONS,
    model = "gpt-4o-mini",
    output_type=WebSearchPlan
)

In [6]:
message = "Latest Agentic AI frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the latest research papers and presentations that discuss agentic AI frameworks developed in 2025.', query='latest agentic AI frameworks 2025'), WebSearchItem(reason='To discover industry reports or news articles focusing on advancements in agentic AI technologies and frameworks released in 2025.', query='agentic AI technology updates 2025')]


In [7]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("uditsdaily@gmail.com") # Change this to your verified email
    to_email = To("uditdgreat@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [8]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email Agent", 
    model = "gpt-4o-mini",
    tools = [send_email],
    instructions=INSTRUCTIONS
)

In [9]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 2-3 pages of content, at least 500 words."
)

class ReportData(BaseModel):
    short_summary : str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report : str = Field(description="The final report")

    follow_up_questions : list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name = "Writer Agent",
    model = "gpt-4o-mini",
    instructions=INSTRUCTIONS,
    output_type=ReportData
)


The next 3 functions will plan and execute the search, using planner_agent and search_agent


In [10]:
async def plan_searches(query: str):
    """Use the planner agent to plan which searches to run for the query"""
    print("Planning Searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in Search Plan"""
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished Working")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in search plan"""
    input = f"Search term: {item.query}\n Reason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

Functions to write report and then mail it

In [11]:
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output


async def send_email(report: ReportData):
    """Use this email agent to send an email with the report"""
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [12]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research Trace"):
    print("Starting Research")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)
    print("Done")


Starting Research
Planning Searches...
Will perform 2 searches
Searching...
Finished Working
Thinking about report...
Finished writing report
Writing email...
Email sent
Done
